<a href="https://colab.research.google.com/github/KolipakaRamesh/AIML_Practice_Excercises/blob/main/Real_Time_Crypto_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers gradio requests pandas plotly

In [2]:
import requests
import pandas as pd

def get_crypto_prices():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 5,
        "page": 1,
        "sparkline": False
    }
    response = requests.get(url, params=params)
    data = response.json()
    df = pd.DataFrame(data)
    return df[["id", "symbol", "current_price", "price_change_percentage_24h"]]


In [3]:
from transformers import pipeline

# Load the Hugging Face summarizer model
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

def generate_summary(df):
    # Build a readable string from the data
    text = ""
    for _, row in df.iterrows():
        text += f"{row['symbol'].upper()} is at ${row['current_price']:.2f} with {row['price_change_percentage_24h']:.2f}% change. "

    # Use the summarizer
    summary = summarizer(text, max_length=50, min_length=20, do_sample=False)[0]["summary_text"]
    return summary


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


In [4]:
import plotly.express as px

def plot_prices(df):
    fig = px.bar(
        df,
        x='symbol',
        y='current_price',
        color='price_change_percentage_24h',
        title='Live Crypto Prices',
        labels={'symbol': 'Crypto', 'current_price': 'Price (USD)'}
    )
    return fig


In [5]:
import gradio as gr

def dashboard():
    df = get_crypto_prices()
    summary = generate_summary(df)
    fig = plot_prices(df)
    return fig, summary

demo = gr.Interface(
    fn=dashboard,
    inputs=[],
    outputs=["plot", "text"],
    title="💹 Real-Time Crypto Dashboard with AI Summary",
    live=True
)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0ce116fb4036aaf0d5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
